In [1]:
!pip install transformers
!pip install torch
!pip install scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.0 MB/s eta 0:00:00


In [11]:
import pandas as pd

# Name of the input Excel file
input_file = '/content/updated_data_on.xlsx'  # Replace with your file name

# Read all sheets from the input Excel file into a dictionary of DataFrames
df = pd.read_excel(input_file)


In [14]:
df = df.fillna("")

In [15]:
df

,ask_howru,ask_howold,ask_isbot,ask_ishuman,ask_whatspossible,ask_languagesbot,ask_fyn,ask_botcreation,Hand_over,customer_support,...,good_morning,good_afternoon,good_evening,good_night,about_onpassive,founder,location,onpassive_profile,onpassive_sucks,out_of_scope
0,Are you ok?,How long have you been alive?,Are you a bot,hey are you human,Can you explain me in one sentence what you ar...,Are you familiar with more than one language?,im fine,wer r u,Can I talk to a real agent?,service,...,hi sona good morning,hi good afternoon,Hi Good evening sona,good night,onpassive,may i know about ceo of onpassive,where is onpassive?,what is the onpassive company profile,what is onpassive sucks,get lost
1,Do you feel good?,How many years have you been alive?,Are you a chat bot?,are u human,Can you help me?,Can you communicate in any other languages?,i'm fyn,wer are you,I would like to talk to a real person.,What are the services you are providing.,...,good morning,hy good aftrn,hy Good eve,gud nght,what is onpassive?,may i know about ceo of onpassive,what's the location of onpassive?,wht is th onpassive company profile,The worst is onpassive.,i want to sucide
2,Do you have a great day?,How old are you?,"Hi there, are you the bot?",r u a human,Could you please show me what you can,Can you speak more than one language?,i'm good,wer r u from,Can I speak to someone?,how to reach customer service,...,hey gud mng,"hai, gudNoon",hai Gud evng,good nite,Tell me about ONPASSIVE?,who is the ceo of onpassive?,where can I find onpassive in hyderabad?,tell about company profile,Onpassive is worst,I am asking you an out of scope question
3,How are you,What is your age?,are you a real bot?,are you real human?,"Great, is there anything else you can do, bot?",Do you know any other languages?,i m good with you,who r you?,I don't want to talk to a bot.,I want to solve my queries,...,gud mrng,hey good noon,"Hey, Good sunset",gn,Tell me about ONPASSIVE?,who is ash?,in which region onpassive is present?,company profile,Onpassive behaviour stinks.,4 + 2 = ?
4,How are things?,What is your birthdate?,are you a robot,you are a human,How can you help me,Do you know other languages?,i m gud,wru?,Can you connect me to your Support?,I need help to solve my queries,...,good mrg,"hello, gud afternoon","Hello, Gud sundown",gud nyt,what does onpassive do?,founder of onpassive?,what is the address of onpassive?,onpassive profile,Ongoing onpassive is bad.,But you're an english site
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,,,,,,,,,,,...,,,,,,,,,,u
421,,,,,,,,,,,...,,,,,,,,,,V
422,,,,,,,,,,,...,,,,,,,,,,z
423,,,,,,,,,,,...,,,,,,,,,,y


In [21]:
import pandas as pd

df = pd.DataFrame(df)

# Initialize a new DataFrame for the transformed data
transformed_df = pd.DataFrame(columns=['index', 'class', 'text'])

# Iterate through the rows and assign values to the 'class' and 'text' columns
for idx, row in df.iterrows():
    for col_name, cell_value in row.items():
        if cell_value is not None:
            transformed_df = transformed_df.append({'index': idx, 'class': col_name, 'text': cell_value}, ignore_index=True)

# Save the transformed DataFrame to a CSV file
transformed_df.to_csv('output_transformed.csv', index=False)


Streaming output truncated to the last 5000 lines.
<ipython-input-21-08cc5f615b51>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transformed_df = transformed_df.append({'index': idx, 'class': col_name, 'text': cell_value}, ignore_index=True)
<ipython-input-21-08cc5f615b51>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transformed_df = transformed_df.append({'index': idx, 'class': col_name, 'text': cell_value}, ignore_index=True)
<ipython-input-21-08cc5f615b51>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transformed_df = transformed_df.append({'index': idx, 'class': col_name, 'text': cell_value}, ignore_index=True)
<ipython-input-21-08cc5f615b51>:12: FutureWarning: The frame.append method is deprecated and will

In [26]:
input_file = '/content/output_transformed.csv'  # Replace with your file name

# Read all sheets from the input Excel file into a dictionary of DataFrames
updated_df = pd.read_csv(input_file)

In [28]:
updated_df = updated_df.dropna()
# Assuming df_cleaned is your DataFrame without NaN values
updated_df.reset_index(drop=True, inplace=True)


In [34]:
updated_df = updated_df.dropna()


In [35]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load and preprocess your raw data
# Load your dataset as a pandas DataFrame
texts = updated_df['text'].tolist()  # Assuming your text data is in a column named 'text_column'
labels = updated_df['class'].tolist()  # Assuming your labels are in a column named 'label_column'

# Label Encoding (converting class labels to numerical values)
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Tokenization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
X_train_encoded = tokenizer(X_train, padding=True, truncation=True, return_tensors='pt', max_length=128)
X_test_encoded = tokenizer(X_test, padding=True, truncation=True, return_tensors='pt', max_length=128)

# Create Data Loaders
train_dataset = TensorDataset(X_train_encoded['input_ids'], X_train_encoded['attention_mask'], torch.tensor(y_train))
test_dataset = TensorDataset(X_test_encoded['input_ids'], X_test_encoded['attention_mask'], torch.tensor(y_test))
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Define the Model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_classes)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

# Training
epochs = 3  # You can adjust this
model.train()
for epoch in range(epochs):
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.tolist())
        all_labels.extend(labels.tolist())

accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy}')



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Accuracy: 0.6116504854368932


In [39]:
updated_df['class'].unique()

array(['ask_howru', 'ask_howold', 'ask_isbot', 'ask_ishuman',
       'ask_whatspossible', 'ask_languagesbot', 'ask_fyn',
       'ask_botcreation', 'Hand_over', 'customer_support',
       'support_feedback', 'contact_details', 'react_positive', 'goodbye',
       'thank', 'language_support', 'assist_me', 'greet', 'good_morning',
       'good_afternoon', 'good_evening', 'good_night', 'about_onpassive',
       'founder', 'location', 'onpassive_profile', 'onpassive_sucks',
       'out_of_scope'], dtype=object)

In [37]:
new_text = "hi"
inputs = tokenizer(new_text, padding=True, truncation=True, return_tensors='pt', max_length=128)

# Inference
with torch.no_grad():
    outputs = model(inputs['input_ids'], attention_mask=inputs['attention_mask'])
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

# If you encoded labels, you can decode the predicted class
# predicted_class = label_encoder.inverse_transform([predicted_class])

print(f"Predicted Class: {predicted_class}")
#Replace 'my_model.pth' with the path to your saved model and adjust the new_text variable with the text you want to classify. If you encoded labels during training and saved the label encoder, you can also decode the predicted class label as shown in the commented lines.


Predicted Class: 19
